# Instructions for Freesurfer

1. first use python to move files a few at a time.

In [42]:
import os 
import shutil
import glob

In [7]:
find /mnt/data/output -iwholename '*Freesurfer*/*.nii.gz' -type f

In [29]:
shutil.copy?

Signature: shutil.copy(src, dst, *, follow_symlinks=True)
Docstring:
Copy data and mode bits ("cp src dst"). Return the file's destination.

The destination may be a directory.

If follow_symlinks is false, symlinks won't be followed. This
resembles GNU's "cp -P src dst".

If source and destination are the same file, a SameFileError will be
raised.
File:      c:\users\candacemoore\miniconda3\envs\layer\lib\shutil.py
Type:      function

In [46]:
glob.glob?

Signature:
glob.glob(
    pathname,
    *,
    root_dir=None,
    dir_fd=None,
    recursive=False,
    include_hidden=False,
)
Docstring:
Return a list of paths matching a pathname pattern.

The pattern may contain simple shell-style wildcards a la
fnmatch. Unlike fnmatch, filenames starting with a
dot are special cases that are not matched by '*' and '?'
patterns by default.

If `include_hidden` is true, the patterns '*', '?', '**'  will match hidden
directories.

If `recursive` is true, the pattern '**' will match any files and
zero or more directories and subdirectories.
File:      c:\users\candacemoore\miniconda3\envs\layer\lib\glob.py
Type:      function

In [58]:
_files = glob.glob('C:/presentations/**', recursive=True)

In [60]:
#_files

In [44]:
for fileName_relative in glob.glob('C:/experimental', *, recursive=True):       ## first get full file name with directores using for loop

    print("Full file name with directories: ", fileName_relative)

SyntaxError: invalid syntax (2802970114.py, line 1)

In [39]:
def find_surf_and_move(src, dst, filestring):
    if not os.path.exists(dst):
        os.mkdir(dst)
    files = glob.glob(os.path.join(src,'**'), recursive=True)

    # for path, subdirs, files in os.walk(src):
    #     print(files)
    #     for name in files:
    #         if filestring in name:
    #         #if fnmatch(name, pattern):
    #             full_name = os.path.join(path, name)
    #             #destination_folder = 
    #             print(name)

                #shutil.copytree(real_name, dst)
    
    
    
    
#     files = os.listdir()
    
#     if filestring is in file:
        
#     shutil.copytree(src, dst) 

In [40]:
find_surf_and_move(file_path, 'C:/experimental', 'cvasl')

['cvasl_review.pptx', 'cvasl_review_4_deeplife.pptx', 'cvasl_review_4_partners.pptx', 'diversity_nlesc_ethics [Autosaved].pptx', 'diversity_nlesc_ethics.pptx', 'fosdem_notes.pptx', 'foss_for_docs.pptx', 'foss_for_docs_f.pdf', 'foss_for_docs_f.pptx', 'intro_diagnostic_imaging [Autosaved].pptx', 'intro_diagnostic_imaging.pptx', 'intro_diagnostic_imaging_nf.pptx', 'lightening_ethics.pptx', 'lightening_ethics_vpu.pptx', 'mip_plea.pptx', 'mip_plea2.pptx', 'reproducability_hackathon.pptx', 'talk_repro_may7th.pptx', 'training_intro_slides.pptx', 'training_intro_slides_image_processing_March_2024.pptx']
cvasl_review.pptx
cvasl_review_4_deeplife.pptx
cvasl_review_4_partners.pptx
['.gitignore', '.readthedocs.yml', 'LICENSE', 'README.rst', 'requirements.txt', 'setup.py', 'setup_patched.py']
['clustering.py', 'neurocombat.py', 'nifti.py', 'utils.py', '__init__.py']
['clustering.cpython-37.pyc', 'neurocombat.cpython-37.pyc', 'nifti.cpython-37.pyc', 'utils.cpython-37.pyc', '__init__.cpython-37.pyc']

In [9]:
root = file_path
#pattern = "*."

for path, subdirs, files in os.walk(root):
    for name in files:
        if 'cvasl' in name:
            #if fnmatch(name, pattern):
            print(os.path.join(path, name))

C:/presentations\cvasl_review.pptx
C:/presentations\cvasl_review_4_deeplife.pptx
C:/presentations\cvasl_review_4_partners.pptx
